# Watch Me Code 3: Mapping with Folium

Folium is a Python wrapper library for the OpenStreetMaps api. It allows you to place data on a map in a variety of ways.


In [1]:
! pip install folium

You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
import folium
import pandas as pd
import random 

In [13]:
# we need to center the map in the middle of the US. I googled for the location.
CENTER_US = (39.8333333,-98.585522)
london = (51.5074, -0.1278)
map = folium.Map(location=CENTER_US, zoom_start=2)
map

## Map Pins

In [8]:
# read in a data file of IP address to locations.
data = pd.read_csv('https://raw.githubusercontent.com/mafudge/datasets/master/clickstream/ip_lookup.csv')
data.sample(5)

,IP,Country,State,City,ApproxLat,ApproxLng
8,128.230.122.180,USA,NY,Syracuse,43.048122,-76.147424
4,155.100.169.152,USA,UT,Salt Lake City,40.760779,-111.891047
13,8.37.70.170,USA,CA,Los Angeles,34.052234,-118.243685
14,8.37.70.77,USA,CA,Los Angeles,34.052234,-118.243685
6,70.209.14.54,USA,FL,Tampa,27.950575,-82.457178


In [11]:
# Let's place each location on the map
for row in data.to_records():
    pos = (row['ApproxLat'],row['ApproxLng'])
    marker = folium.Marker(location=pos, 
                    popup="%s, %s" % (row['City'],row['State'])
                          )
    map.add_children(marker)
map

/Users/eldon/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  import sys


In [10]:
# Same thing with a different icon and colors. Icons come from http://fontawesome.io/icons/ but its an older version.
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
            'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
            'darkpurple', 'pink', 'lightblue', 'lightgreen',
            'gray', 'black', 'lightgray']
for row in data.to_records():
    pos = (row['ApproxLat'],row['ApproxLng'])
    marker = folium.Marker(location=pos, 
                    popup="%s, %s" % (row['City'],row['State']),
                    icon = folium.Icon(color = random.choice(colors), icon='user')
                          )
    map.add_children(marker)
map

/Users/eldon/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  if sys.path[0] == '':


In [14]:
# There are other map tiles available. See https://folium.readthedocs.io/en/latest/quickstart.html
# Instead of Markers we use circles colors are HTML color codes http://htmlcolorcodes.com/
CENTER_US = (39.8333333,-98.585522)
map2 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Stamen Toner')
for row in data.to_records():
    map2.add_children(folium.CircleMarker(location=(row['ApproxLat'],row['ApproxLng']), 
                    popup=row['City'], radius=10000, color='#0000FF',  fill_color='#FF3333'))
    
map2

/Users/eldon/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  import sys


# Choropleths

Choropleths are cartographic overlays based on boundries defined in a geo JSON file.


In [15]:
# State level geo-json overlay choropleth
CENTER_US = (39.8333333,-98.585522)
state_geojson = 'WMC3-us-states.json'
map3 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
map3.choropleth(geo_path=state_geojson)
map3


TypeError: choropleth() got an unexpected keyword argument 'geo_path'

In [9]:
states = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')
state_counts = pd.DataFrame( {'Counts' : data['State']. value_counts() } ).sort_index()
state_counts['StateCode'] = state_counts.index
state_data = states.merge(state_counts, how="left", left_on='Abbreviation', right_on='StateCode')
state_data = state_data[['Abbreviation','Counts']]
state_data = state_data.fillna(0)
state_data


,Abbreviation,Counts
0,AL,0.0
1,AK,0.0
2,AZ,0.0
3,AR,0.0
4,CA,10.0
5,CO,0.0
6,CT,0.0
7,DE,0.0
8,DC,0.0
9,FL,1.0


In [16]:
CENTER_US = (39.8333333,-98.585522)
state_geojson = 'WMC3-us-states.json'
map3 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
map3.choropleth(geo_path=state_geojson,data=state_data, columns=['Abbreviation','Counts'], 
                key_on ='feature.id', fill_color='BuGn', legend_name='Website Visitors')
map3

NameError: name 'state_data' is not defined

In [17]:
# Here's a more straigtforward example with unemployment data:
unemployment = pd.read_csv('https://raw.githubusercontent.com/wrobstory/vincent/master/examples/data/US_Unemployment_Oct2012.csv')
state_geojson = 'WMC3-us-states.json'
map4 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
map4.choropleth(geo_path=state_geojson,data=unemployment, 
                columns=['State','Unemployment'], key_on ='feature.id', fill_color='YlOrRd', legend_name='2012 US Unemployment')
map4


TypeError: choropleth() got an unexpected keyword argument 'geo_path'